In [ ]:
## path to the datasets
datadir='../Datasets/'

## required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import igraph as ig
import partition_igraph
from collections import Counter
import random

# Part 2 - Clustering

First, let's build the airport graph as we did in Part 1.

In [ ]:
## the airports (nodes)
airport_df = pd.read_csv(datadir + 'Airports/airports_loc.csv')
airport_df.head()

## read edges from csv file
df_edges = pd.read_csv(datadir + 'Airports/connections.csv')
df_edges.head() ## look at a few edges

## build directed graph
tuple_list = [tuple(x) for x in df_edges.values]
g = ig.Graph.TupleList(tuple_list, directed=True, edge_attrs=['weight'])

## add vertex attributes
lookup = {k:v for v,k in enumerate(airport_df['airport'])}
l = [lookup[x] for x in g.vs()['name']]
g.vs['layout'] = [(airport_df['lon'][i],-airport_df['lat'][i]) for i in l]
g.vs['state'] = [airport_df['state'][i] for i in l]
g.vs['city'] = [airport_df['city'][i] for i in l]
g.vs['color'] = 'lightblue'

## undirected graph w/o loops
g_und = g.as_undirected(combine_edges=sum)
g_und = g_und.simplify(combine_edges=sum)


## 2.1 Triangles and Transitivity

Edges represent relations between entities (nodes) in graphs/networks.

The next step is to consider **triads** of nodes 

Fully connected triads form **triangles**

The presence of triangles is indicative of communities (dense subgraph(s) in a graph): ''the friend of my friend is my friend''.

Two fundamental measures of the presence of triangles in graphs are:

* **transitivity** (global clustering coefficient) measures the proportion of wedges (two-hop path in an undirected graph) that form a triangle
* **local transitivity** (local clustering coefficient) for a node is the proportion of pairs of neighbours that form a triangle
  * nodes of degree less than 2 are either ignored or given value 0
  * **average local transitivity** is obtained by averaging local transitivity over all nodes

These measures of transitivity assume **undirected** graphs.

While we can define triangles and other motifs for directed graphs, clustering generally assumes undirected graphs.

Let's first look for global and average local transitivity.

In [ ]:
print('transitivity (global):', g_und.transitivity_undirected())
print('avg local trans:', g_und.transitivity_avglocal_undirected(mode='zero'))

By setting `mode='zero'`, in `.transitivity_avglocal_undirected()`, nodes with degree < 2 were given 0 local transitivity. `mode` defines how to treat vertices with degree less than two. The default setting is `mode='nan'`, in which case vertices will be excluded from the average.

Now for each node's local transitivity, and then averaging manually

In [ ]:
g_und.vs['trans'] = g_und.transitivity_local_undirected(mode='zero') 
print('avg local trans:', np.mean(g_und.vs['trans']))

Let's look at a high transitivity example. Here we see that all neighbour pairs are linked by an edge.

In [ ]:
v = np.argmax(g_und.vs['trans'])
print('airport:',g_und.vs[v]['name'],', transitivity',g_und.vs[v]['trans'],', plotting its ego-net:')
g_und.vs[v]['color'] = 'pink'
sg = g_und.subgraph(g_und.neighborhood(v))
ig.plot(sg,bbox=(400,300), vertex_label=sg.vs['name'], vertex_size=15, layout=sg.vs['layout'],
            vertex_label_size=6, margin=50)


Example with zero local transitivity. Here we see that no neighbour pairs are linked by an edge.

In [ ]:
v = [v.index for v in g_und.vs if v['trans'] == 0 and g_und.degree(v)>=3][0]
print('airport:',g_und.vs[v]['name'],', transitivity',g_und.vs[v]['trans'],', plotting its ego-net:')
g_und.vs[v]['color'] = 'pink'
sg = g_und.subgraph(g_und.neighborhood(v))
ig.plot(sg,bbox=(400,300), vertex_label=sg.vs['name'], vertex_size=15, layout=sg.vs['layout'],
            vertex_label_size=6, margin=50)


### Directed graphs

For directed graphs, there are 16 different possibilities for each **triad**

The **triad census** function counts all occurrences, and returns the counts in a specific ordering:
```
  - C{003} -- the empty graph
  - C{012} -- a graph with a single directed edge (C{A --> B, C})
  - C{102} -- a graph with a single mutual edge (C{A <-> B, C})
  - C{021D} -- the binary out-tree (C{A <-- B --> C})
  - C{021U} -- the binary in-tree (C{A --> B <-- C})
  - C{021C} -- the directed line (C{A --> B --> C})
  - C{111D} -- C{A <-> B <-- C}
  - C{111U} -- C{A <-> B --> C}
  - C{030T} -- C{A --> B <-- C, A --> C}
  - C{030C} -- C{A <-- B <-- C, A --> C}
  - C{201} -- C{A <-> B <-> C}
  - C{120D} -- C{A <-- B --> C, A <-> C}
  - C{120U} -- C{A --> B <-- C, A <-> C}
  - C{120C} -- C{A --> B --> C, A <-> C}
  - C{210C} -- C{A --> B <-> C, A <-> C}
  - C{300} -- the complete graph (C{A <-> B <-> C, A <-> C})
```

#### Example: the "300" triad

In [ ]:
ig.plot(ig.Graph.TupleList([('A','C'),('C','A'),('B','C'),('C','B'),('B','A'),('A','B')], 
                           directed=True), bbox=(150,150), edge_curved=0.2, edge_arrow_size=.7)

#### Example: the "030C" triad

In [ ]:
ig.plot(ig.Graph.TupleList([('A','C'),('C','B'),('B','A')], 
                           directed=True), bbox=(150,150), edge_arrow_size=.7)

#### Example: The Airport Graph

In [ ]:
tc = g.triad_census()
print('number of triads:',np.sum(tc))
print('number of complete subgraphs (300):',tc.t300) 
print('number of 3-edge cycles (030C)',tc.t030C)
print('complete list by type:',tuple(tc))

### Cliques

Triangles are also known as **3-cliques**

A **k-clique** is a fully connected subgraph with k nodes

The **clique number** is the size of the largest clique.

Let's explore the cliques in the Airport graph

In [ ]:
print('number of 3-cliques:', len(g_und.cliques(min=3, max=3)))
print('number of 4-cliques:', len(g_und.cliques(min=4, max=4)))
print('max clique size:', g_und.clique_number())

There are many cliques with max clique size.

In [ ]:
len(g_und.cliques(min=38))

Print some airports in a max clique. These are all major hubs.

In [ ]:
[g_und.vs[v]['name'] for v in g_und.cliques(min=38)[0][:10]]

### Questions

#### 1. Find the node in the undirected airport graph with degree 5 or more having the lowest transitivity.


#### 2. Plot its ego-net, what do you observe?


#### 3. Compute its betweenness, compare with average betweenness for all nodes. How would you interpret this result?

### Possible Solutions

In [ ]:
## node with low transitivity, degree at least 5
x = np.argmin([v['trans'] for v in g_und.vs if g_und.degree(v)>=5])
v = [v for v in g_und.vs if g_und.degree(v)>=5][x]
v['color'] = 'pink'
print('airport:',v['name'],', transitivity',v['trans'],', plotting its ego-net:')
sg = g_und.subgraph(g_und.neighborhood(v))
ig.plot(sg,bbox=(400,300), vertex_label=sg.vs['name'], vertex_size=15,layout=sg.vs['layout'],
            vertex_label_size=6, margin=50)

## We see that very few pairs of neighhbours are connected


In [ ]:
## compare its betweenness with average node beteweenness
g_und.betweenness(v) / np.mean(g_und.betweenness())

## it has high betweenness ... 
## with several neighbours not linked by an edge, this node is on several geodesics!


In [ ]:
## clean-up
del(g_und.vs['trans'])
g_und.vs['color'] = 'lightblue'

## 2.2 Clustering

Graph clustering, a.k.a. **node partitioning**, is a very active research area, with dozens of algorithms.

Some good ones are:

* Louvain (multilevel): 
    * fast, but may return disconnected communities
    * unstable for graphs with homogeneous edge weights
* Leiden:
    * faster than Louvain, connected communities
* ECG (ensemble clustering):
    * better stability for graphs with homogeneous edge weights

Measures of community strength include:
* modularity ("proportion of edges within communities" - "expected proportion under null model")
* comparing degree within and between communities


### Clusters in the airport graph

We're going to use the Leiden algorithm. Since it's a randomized algorithm, let's fix a random seed so we can compare results later.

In [ ]:
random.seed(31416)

Cluster and color nodes with respect to communities

In [ ]:
g_leiden = g_und.community_leiden(objective_function='modularity', weights='weight')
## assign colors to different communities
pal = ig.ClusterColoringPalette(n=np.max(g_leiden.membership)+1)
g_und.vs['color'] = [pal.get(i) for i in g_leiden.membership]

Size of each cluster found

In [ ]:
g_leiden.sizes()

By visualizing, we see that the 4 big clusters seem related to geographical locations

In [ ]:
ig.plot(g_und, vertex_size=5, edge_color='grey', layout=g.vs['layout'], bbox=(500,400))

Display one (small) cluster

In [ ]:
sg = g_leiden.subgraph(6)
## closely connected airports in 3 states: WY, NE and ND
print(Counter(sg.vs['state']))
ig.plot(sg, bbox=(400,300), vertex_label_size=8, vertex_label=sg.vs['name'], 
        layout=sg.vs['layout'], )

Compute modularity (<=1); large positive values of modularity are indicative of community structure. The result here is not very high (more on this later).

In [ ]:
g_und.modularity(g_leiden.membership, weights='weight')

Collapse communities to show degree between and within communities

In [ ]:
g_und.es['label'] = 1
g_und.vs['lat'] = [v['layout'][0] for v in g_und.vs]
g_und.vs['lon'] = [v['layout'][1] for v in g_und.vs]
g_collapse = g_leiden.cluster_graph(combine_vertices={'lat':np.mean,'lon':np.mean, 'color':'first'}, combine_edges={'label':sum})
g_collapse.vs['label'] = [2*G.ecount() for G in g_leiden.subgraphs()]
ly = [(v['lat'],v['lon']) for v in g_collapse.vs]
ig.plot(g_collapse, layout=ly, bbox=(450,350), vertex_label_size=8, edge_label_size=8)

#### Observation

From the above, we see that the small community we plotted earlier (airports from WY, NE and ND) has connections with the 4 main clusters; this and the fact that is tightly connected internally explains why a modularity-based algorithm will group those in their own cluster.

We also observe a large number of edges between the main 4 communities, thus the relatively low modularity value we observed.

Having a few small communities (here with 2 nodes) camn happen for various reasons; here, we see an equal number of edges toward 2 large communities, so it is likely that there is no gain in madularity moving those nodes to one of those communities.
Being is a separate component is also a reason not to cluster with other nodes.


In [ ]:
## clean up
del(g_und.es['label'])
del(g_und.vs['lat'])
del(g_und.vs['lon'])

### Questions

#### 1. What are the most frequent states in each of the clusters we found?


#### 2. Build the subgraph with airports from Hawai (HI) and Florida (FL) only, delete nodes with zero degree and cluster with Leiden; what do you get?


### Possible Solutions

In [ ]:
## Leiden - most frequent states
for sg in g_leiden.subgraphs():
    print('cluster:',sg.vcount(),'nodes, frequent states:',Counter(sg.vs['state']).most_common(5))


In [ ]:
## Build HI-FL subgraph
sg = g_und.subgraph([v for v in g_und.vs if v['state'] in {'HI','FL'}])
sg.delete_vertices([v.index for v in sg.vs if sg.degree(v)==0])
## Cluster
sg_cl = sg.community_leiden(objective_function='modularity', weights='weight')
## plot
pal = ig.ClusterColoringPalette(n=np.max(sg_cl.membership)+1)
sg.vs['color'] = [pal.get(i) for i in sg_cl.membership]
ig.plot(sg, vertex_size=5, edge_color='grey', layout=sg.vs['layout'], bbox=(300,200))


## 2.3 Random Graph Models

Random graph models are useful for various reasons:
* interpretation of results on real graphs (ex: is this value expected? high? low?)
* to compare algorithms (ex: clustering algorithms)
* to study theoretical properties

Usually, we fix some aspects of the graph, such as the **number of nodes and edges**, and randomly sample.

There are many such models, including:
* Erdos-Renyi model: fix the number of nodes and edges, and randomly place edges
* Configuration model: as above, but given a degree distribution for the nodes
* ABCD model: power-law node degree distribution with community structure


### Compare transitivity and other community values for a random graph

Let's build an Erdos-Renyi (ER) graph with same number of nodes/edges as the (undirected) airport graph

In [ ]:
random.seed(31416) ## for better reproducibility
g_er = ig.Graph.Erdos_Renyi(n=g_und.vcount(), m=g_und.ecount())
print('min degree',np.min(g_er.degree()),'max degree',np.max(g_er.degree()))

Let's check that it has the same number of nodes and edges as the airport graph

In [ ]:
(g_er.vcount() == g_und.vcount()) & (g_er.ecount() == g_und.ecount())

Compare global transitivity for the random (ER) and airport graphs.


In [ ]:
print('transitivity (global), airport graph:    ',g_und.transitivity_undirected())
print('transitivity (global), random (ER) graph:',g_er.transitivity_undirected())

Number of cliques in the random (ER) graph and clique number.
All values much smaller than for the airport graph!


In [ ]:
print('Airport graph:')
print('number of 3-cliques:', len(g_und.cliques(min=3, max=3)))
print('number of 4-cliques:', len(g_und.cliques(min=4, max=4)))
print('max clique:', g_und.clique_number(),'\n')

print('Random (ER) graph:')
print('number of 3-cliques:', len(g_er.cliques(min=3, max=3)))
print('number of 4-cliques:', len(g_er.cliques(min=4, max=4)))
print('max clique:', g_er.clique_number())

Modularity for Leiden communities in the random (ER) graph and comparison with the airport graph.

In [ ]:
print('Modularity, airport graph with Leiden:    ',g_und.modularity(g_leiden.membership, weights='weight'))
er_leiden = g_er.community_leiden(objective_function='modularity')
print('Modularity, random (ER) graph with Leiden:',g_er.modularity(er_leiden.membership))

Cluster sizes, random (ER) graph

In [ ]:
er_leiden.sizes()

Let's look at one of the community subgraphs

In [ ]:
er_sg = er_leiden.subgraph(5)
ig.plot(er_sg, bbox=(400,300), vertex_size=8)

### Question

#### 1. Generate a random graph with the same degree distribution  as the undirected airport graph. This is known as a "configuration model". Check that it's degree distribution matches the undirected airport graph. How do the number of edges and nodes compare?

Hint: for the configuration model, use: ```ig.Graph.Degree_Sequence()``` with the proper degree distribution.


#### 2. Compute the global transitivity, number of 3 and 4 cliques and clique number for this random graph.

#### 3. Cluster the random graph with Leiden (using modularity as objective) and compute resulting modularity

#### 4. Interpret the findings about the airport graph when comparing it's structure to the Erdos-Renyi (ER) and configuration model random graphs with the same number of nodes/edges with respect to:
 * Transitivity
 * Clique distribution
 * Clustering/modularity

### Possible solutions

In [ ]:
random.seed(31416) ## for better reproducibility

## 1. build random graph
g_degseq = ig.Graph.Degree_Sequence(g_und.degree())
print('degree distributions match:', g_und.degree() == g_degseq.degree())
print('number of nodes match:', g_und.vcount() == g_degseq.vcount())
print('number of edges match:', g_und.ecount() == g_degseq.ecount())

## 2. transitivity, cliques
print('global transitivity:',g_degseq.transitivity_undirected())
print('number of 3-cliques:', len(g_degseq.cliques(min=3, max=3)))
print('number of 4-cliques:', len(g_degseq.cliques(min=4, max=4)))
print('max clique:', g_degseq.clique_number())

## 3. clustering and modularity
g_degseq_leiden = g_degseq.community_leiden(objective_function='modularity')
print('modularity:',g_degseq.modularity(g_degseq_leiden.membership))




#### Observations

The configuration model (degree sequence) has higher transitivity and more cliques than ER, but still less than the reference (airport) graph.
Modularity for both random models is lower than the reference graph.


## Bonus Material
See the "extra" notebook for Part 2.

# To go further

More topics can be found in:
* Book: https://www.ryerson.ca/mining-complex-networks
* Notebooks: https://github.com/ftheberge/GraphMiningNotebooks
    
These references include:   
* more centrality measures
* clustering: overlapping clusters, outliers    
* degree assortativity
* vertex and graph embedding
* hypergraphs
* network robustness
* road networks
